In [7]:
import pandas as pd
import numpy as np
import math

import pymysql

In [8]:
MONTH = 3
K = 20

In [9]:
# POI_clustering으로 저장한 POI들 load
locations = pd.read_csv(f'../data/2021_07_POI_{K}_transformed.csv')

locations = locations[["lon", "lat"]]
locations.index = [f'POI{i}' for i in range(len(locations))]

In [10]:
df = pd.read_csv(f"../data/month_{MONTH}.csv")

# csv파일로 불러온 시간(Type : str)을 datetime으로 변경(이후 코드에서 datetime 사용)
df['collection_dt'] = pd.to_datetime(df['collection_dt'])
df.sort_values(by='collection_dt', ascending=True)

,oid,collection_dt,longitude,latitude
2775,46100670,2020-03-07 05:00:03.584,126.511299,33.481273
2776,46100670,2020-03-07 05:00:33.588,126.508891,33.481293
2777,46100670,2020-03-07 05:01:03.593,126.505322,33.481422
2778,46100670,2020-03-07 05:01:33.597,126.501410,33.481574
2779,46100670,2020-03-07 05:02:03.601,126.500675,33.480566
...,...,...,...,...
2397273,4610179f,2021-03-28 04:59:00.946,126.665856,33.542204
2397274,4610135c,2021-03-28 04:59:30.074,126.489862,33.483623
2397275,4610179f,2021-03-28 04:59:30.512,126.665793,33.542190
2397276,46101027,2021-03-28 04:59:30.621,126.358063,33.252910


## GPS 데이터로 Trajectory 만들기
    trajectory : (trajectory_id, start_point, end_point, path, time_period)

In [11]:
# Haversine 공식 : 위도, 경로 간 거리 구하기
def _haversine_distance(lon1, lat1, lon2, lat2):
    R = 6371  # 지구의 반지름 (단위: km)
    lon1_rad = math.radians(lon1)
    lat1_rad = math.radians(lat1)
    lon2_rad = math.radians(lon2)
    lat2_rad = math.radians(lat2)
    
    diff_lon = lon2_rad - lon1_rad
    diff_lat = lat2_rad - lat1_rad

    a = math.sin(diff_lon/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(diff_lat/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c

    return distance

In [12]:
def _map_time_period(time):
    if time < pd.to_datetime('06:00:00').time():
        return '새벽'
    elif time < pd.to_datetime('12:00:00').time():
        return '오전'
    elif time < pd.to_datetime('18:00:00').time():
        return '오후'
    else:
        return '저녁'

In [13]:
def _calculate_group_distance(rows):
    # grouping한 데이터의 rows(데이터프레임)의 distance와 velocity 계산
    long  = rows['longitude'].values
    lat  = rows['latitude'].values
    time_delta = rows['collection_dt'].diff().dropna().values

    # 단위 : km
    dist = [_haversine_distance(long[i], lat[i], long[i+1],lat[i+1]) for i in range(len(rows) -1)]    
    
    dist.insert(0, np.NaN)

    rows['distance'] = dist
    
    rows['time_period'] = rows['collection_dt'].dt.time.apply(_map_time_period)
    return rows 

In [14]:
# Haversine 공식을 이용한 Trajectory 생성
# 관심지점 - GPS 데이터 간 거리 1km 이내
def _calculate_path(rows, locations):
    path = []
    for df_index, row in rows.iterrows():
        GPS_lon = row['longitude']
        GPS_lat = row['latitude']
        
        # POI_name : POIx
        distance = [_haversine_distance(GPS_lon, GPS_lat, POI_lon, POI_lat) for POI_name, (POI_lon, POI_lat) in locations.iterrows()]
        nearest_idx = np.argmin(distance)
        # # 1km이하인 지점이 아니라 제일 가까운 POI로 수정해야함
            # if distance <= 1:
            #     path.append(POI_name)
            #     break
        path.append(locations.index[nearest_idx])
    return path

In [15]:
def _remove_duplicated_path(path, cal_rows):
    removed_path = []
    path_time_period = []

    prev_location = path[0]
    start_idx = 0
    end_idx = len(path)-1

    for idx, location in zip(range(len(path)), path):
        
        if location != prev_location:
            end_idx = idx

            same_POI_rows = cal_rows.iloc[start_idx:end_idx]
        
            period_time = same_POI_rows['time_period'].value_counts().idxmax()
            prev_location = location

            
            removed_path.append(location)
            path_time_period.append(period_time)
            
            prev_location = location
            start_idx = idx
        
    return removed_path, path_time_period

In [16]:
# Haversine 공식 : 위도, 경로 간 거리 구하기
def _haversine_distance(lon1, lat1, lon2, lat2):
    R = 6371  # 지구의 반지름 (단위: km)
    lon1_rad = math.radians(lon1)
    lat1_rad = math.radians(lat1)
    lon2_rad = math.radians(lon2)
    lat2_rad = math.radians(lat2)
    
    diff_lon = lon2_rad - lon1_rad
    diff_lat = lat2_rad - lat1_rad

    a = math.sin(diff_lon/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(diff_lat/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c

    return distance

In [17]:
# DataFrame을 (년, 월, 일, oid)로 grouping
grouped = df.groupby([df['collection_dt'].dt.year, 
                        df['collection_dt'].dt.month, 
                        df['collection_dt'].dt.day, 
                        df['oid']])

trajectories = pd.DataFrame(columns=['trajectory_id', 'start_point','end_point', 'path', 'time_period'])
concat_row = pd.DataFrame(columns=['trajectory_id', 'start_point','end_point', 'path', 'time_period'])


index = 0

for group_key, rows in grouped:
    rows.drop_duplicates(subset=['collection_dt'], inplace=True)
    if index == 10:
        break
    if (len(rows) <= 100):
        continue
    # rows
    # 타입 : dataframe
    # column : ['oid', 'collection_dt', 'longitude','latitude', 'distance', 'velocity']
    # value example : 769545(df_index), 0c0000fd, 2020-01-04 06:14:10, 126.241423, 33.394894, 0.009780, 29.072355 
    # 첫 row의 distance와 velocity는 NaN으로 할당
    cal_rows = _calculate_group_distance(rows)
    path = _calculate_path(cal_rows, locations)
    
    removed_path, path_time_period = _remove_duplicated_path(path, cal_rows)
    
    # 경로가 너무 길거나 0이면 추가 x
    if (len(removed_path) == 0) or (len(removed_path) >= 50):
        continue

    # 타입 : dictionary 
    # Key 형식 example : (2020, 1, 4, '46100c11')
    # value 형식 example : ['POI1', 'POI3', 'POI5', 'POI47']
    start_point, end_point = removed_path[0], removed_path[-1]

    column_range = ['trajectory_id', 'start_point', 'end_point', 'path', 'time_period']
    concat_row.loc[0, column_range] = [ group_key, start_point, end_point, removed_path, path_time_period]
    
    trajectories = pd.concat([trajectories, concat_row], ignore_index=True)
    index += 1
    

## 생성한 Trajectory DataFrame을 Mysql에 저장 

In [18]:
# DB 정보
host = "localhost"
user = "root"
password = "@tjdan34"

if MONTH in [3, 4, 5]:
    db_name = "spring_db"
    table_name = "spring_table"
elif MONTH in [6, 7, 8]:
    db_name = "summer_db"
    table_name = "summer_table"
elif MONTH in [9, 10, 11]:
    db_name = "fall_db"
    table_name = "fall_table"
elif MONTH in [12, 1, 2]:
    db_name = "winter_db"
    table_name = "winter_table"

In [22]:
trajectories = trajectories.astype(str)

conn = pymysql.connect(host=host, user=user, password=password, db=db_name)
curs = conn.cursor(pymysql.cursors.DictCursor)

try:
    # 테이블이 이미 존재하는지 확인
    table_exists = False
    cursor = conn.cursor()
    cursor.execute(f"SHOW TABLES LIKE '{table_name}'")
    result = cursor.fetchone()
    
    if result:
        table_exists = True

    # 테이블이 존재하지 않는 경우에만 CREATE TABLE 문 실행
    if not table_exists:
        create_table_sql = f"""CREATE TABLE `{db_name}`.`{table_name}` (
            `trajectory_id` VARCHAR(500) NOT NULL,
            `start_point` VARCHAR(500) NULL,
            `end_point` VARCHAR(500) NULL,
            `path` VARCHAR(500) NULL,
            `time_period` VARCHAR(500) NULL,
            PRIMARY KEY (`trajectory_id`));"""
        cursor.execute(create_table_sql)

    # INSERT INTO 문 실행
    insert_sql = f"""INSERT INTO {table_name} (trajectory_id, start_point, end_point, path, time_period) VALUES (%s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE start_point=VALUES(start_point), end_point=VALUES(end_point), path=VALUES(path), time_period=VALUES(time_period)"""
    for idx in range(len(trajectories)):
        try:
            curs.execute(insert_sql, tuple(trajectories.values[idx]))
        except Exception as e:
            # Handle other exceptions, if any
            print(f"Error occurred at index {idx}: {e}")
            continue

    # 커밋
    conn.commit()

except Exception as e:
    print(f"Error: {e}")
finally:
    curs.close()
    conn.close()